In [1]:
import pandas as pd
import numpy as np
from enroute_utilities.graph import *
from enroute_utilities.matrix import *
from enroute_utilities.sumo import *
from enroute_utilities.xml import *
from os.path import join as path_join, normpath as norm_path, isfile as is_file
import networkx as nx
from matplotlib import pyplot as plt
import pickle
from scipy.stats import lognorm
from math import exp
from scipy.optimize import nnls
from lxml import etree

to export to python script use below.

In [ ]:
#from timeit import default_timer as timer

In [2]:
origin_path = norm_path(r'C:\Users\Roozbeh\Sumo\London\overlay\50 cameras') #path were sampled data are

In [3]:
csv_extension = ".csv"
txt_extension = ".txt"
camera_input_file_path = norm_path(path_join(origin_path,"london_50_cameras"+csv_extension))
camera_dist_file_path = norm_path(path_join(origin_path,"london_50_camera_distances"+csv_extension))
flows_data_file_path = norm_path(path_join(origin_path,"london_50_cameras_30min_data"+csv_extension))

In [4]:
camerasdf = pd.read_csv(camera_input_file_path)[["CAMERA_ID","EASTING","NORTHING","GEO_LAT","GEO_LON"]]
distdf = pd.read_csv(camera_dist_file_path)[["CAMERA_ID_FROM","CAMERA_ID_TO","TIME","DISTANCE"]]

DEFINE INPUT PARAMETERS

In [5]:
no_of_cameras = len(camerasdf)

In [ ]:
K =  #number of shortest paths ot use

In [ ]:
mu = 7
sigma = 1.4

In [ ]:
camerasdf.describe()

In [ ]:
camerasdf[:2]

In [ ]:
distdf["TIME"] = distdf["TIME"].str.replace("[^0-9]+", "").astype(np.int)

In [ ]:
%%time
matrix = get_distance_matrix(distdf,camerasdf)

In [ ]:
matrix

In [ ]:
G = nx.from_numpy_matrix(matrix)

In [ ]:
#%matplotlib inline
#nx.draw_shell(G)

In [ ]:
kshortest_paths_dump_path = norm_path(path_join(origin_path,str(no_of_cameras)+"_cameras_"+str(K)+"_shortest_paths.dump"))

In [ ]:
%%time
if (is_file(kshortest_paths_dump_path)):
    print "K Shortest path dump already exists. Loading!"
    a = pickle.load(open(kshortest_paths_dump_path,"r"))
else:
    print "K Shortest path dump not existed. Generate and dumping to file! It might take a while!"
    a = k_shortest_paths(no_of_cameras,K,G)
    pickle.dump(a,open(kshortest_paths_dump_path,"w"))

In [ ]:
a[0][1]

In [ ]:
#To measure the time it takes to generate 5 shortest paths (I believe in seconds!):
#print end_time - start_time

In [ ]:
lnpdf = lambda x: lognorm.pdf(x, s = sigma, scale = exp(mu)/60)
lncdf = lambda x: lognorm.cdf(x, s = sigma, scale = exp(mu)/60)

In [ ]:
%matplotlib inline

In [ ]:
x = [i*0.1 for i in range(1000)]
plt.plot(x,lnpdf(x))

A_matrix generation

In [ ]:
%%time
A_matrix = get_A_matrix(a,lnpdf,no_of_cameras)

In [ ]:
A_matrix

In [ ]:
A_matrix_dump_addr = norm_path(path_join(origin_path,str(no_of_cameras)+"_cameras_A_matrix"))

In [ ]:
np.savetxt(A_matrix_dump_addr+txt_extension,A_matrix,fmt="%3.8f")
np.save(A_matrix_dump_addr,A_matrix)

In [ ]:
flowsdf = pd.read_csv(flows_data_file_path)

In [ ]:
flowsdf.columns = ["Detector","Flow"]

In [ ]:
%%time
b = np.array(get_b_vector_lognorm(flowsdf,distdf,lnpdf)) * 30 #we use lognormal results

In [ ]:
b

In [ ]:
%%time
x,rnorm = nnls(A_matrix,b)

In [ ]:
x

In [ ]:
sum(x)

In [ ]:
x_vector_dump_addr = norm_path(path_join(origin_path,str(no_of_cameras)+"_cameras_A_matrix"))

In [ ]:
np.savetxt(x_vector_dump_addr+txt_extension,x)
np.save(x_vector_dump_addr,x)

In [ ]:
net_file_path = norm_path(path_join(origin_path,"../../london.net.xml"))
camera_with_closest_edge_file_path = norm_path(path_join(origin_path,"london_50_cameras_with_closest_edge"+csv_extension))

In [ ]:
%%time
if is_file(camera_with_closest_edge_file_path):
    print "Camera file with closest edge already exists! Loading!"
    camerasdf = pd.read_csv(camera_with_closest_edge_file_path)[["CAMERA_ID","EASTING","NORTHING","GEO_LAT","GEO_LON","CLOSEST_EDGE"]]
else:
    print "Camera file with closest edge not found! Generate and Writing the file! This might take a while"
    net_file,cameradf["CLOSEST_EDGE"] = get_closest_edge(net_file_path,camerasdf) #if we have the net object we can pass it on as third param so it doesn't load again
    camerasdf.to_csv(camera_with_closest_edge_file_path,index=False)

In [ ]:
taz_root = get_taz_xml_root(camerasdf) #camerasdf should have closest edge

In [ ]:
taz_file_path = norm_path(path_join(origin_path,"london.taz.xml"))

In [ ]:
with open(taz_file_path, 'w') as f:
    f.write(etree.tostring(taz_root,pretty_print=True))

In [ ]:
od_file_path = norm_path(path_join(origin_path,"london.od.txt"))

In [ ]:
with open(od_file_path, "w") as f:
    lines = get_formatted_OD(x,camerasdf,no_of_cameras)
    for line in lines:
        f.write(line+"\n")

Use OD2Trips and them DUAROUTER

C:\Users\Roozbeh\Sumo\London\overlay>od2trips -n london.taz.xml -d london.od.txt -o london.trip.xml

C:\Users\Roozbeh\Sumo\London\overlay>duarouter -n ../../london.net.xml -t london.trip.xml -o london.rou.xml --routing-threads 4